In [ ]:
import os, sys
sys.path.append(os.getcwd().split("concepts")[0][:-1])
# C:\Users\2me41\Company_Short\Projects\linkedin_bot\concepts
os.environ['BASE_PATH'] = "C:/Users/2me41/Company_Short/Projects/linkedin_bot"
os.environ['CHROME_DRIVER_PATH'] = 'C:/Users/2me41/Downloads/chromedriver_win32(3)/chromedriver.exe'

import re
import time
import random
import json
import zipfile
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth


def proxy_extension_maker(PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS, path):
    manifest_json = """
    {
        "version": "1.0.0", "manifest_version": 2, "name": "Chrome Proxy",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking" ],
        "background": { "scripts": ["background.js"] },
        "minimum_chrome_version":"22.0.0"}
    """

    background_js = """
    var config = {
            mode: "fixed_servers",
            rules: {
            singleProxy: {
                scheme: "http",
                host: "%s",
                port: parseInt(%s) },
            bypassList: ["localhost"]  } };

    chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

    function callbackFn(details) {
        return {
            authCredentials: {
                username: "%s",
                password: "%s" } }; }

    chrome.webRequest.onAuthRequired.addListener(
                callbackFn,
                {urls: ["<all_urls>"]},
                ['blocking']
    );
    """ % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)
    
    with zipfile.ZipFile(path, 'w') as zp:
            zp.writestr("manifest.json", manifest_json)
            zp.writestr("background.js", background_js)
            
    return path

class SeleniumBot:
    def __init__(self, proxy_str=None, screen_size=(1920, 1080), headless=False, executable_path="/home/user/Projects/linkedin_bot/data/selenium_driver/chromedriver_linux64/chromedriver"):
        """
        Verify the quality from: https://bot.sannysoft.com/
        verify ip with https://httpbin.org/ip
        """
        
        """options = { 'enable_har': True,  # Capture HAR data, retrieve with driver.har, 'suppress_connection_errors': False }"""
        # self.display = Display(visible=1, size=screen_size)
        # self.display.start()

        capabilities = webdriver.DesiredCapabilities.CHROME

        chrome_options = webdriver.ChromeOptions()
        # chrome_options.add_argument("start-maximized")
        
        # chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        # chrome_options.add_experimental_option('useAutomationExtension', False)

        # Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36

        if proxy_str:
            username, password, host, port = re.split("[\:\@]", proxy_str)
            chrome_options.add_extension(proxy_extension_maker(PROXY_HOST=host, PROXY_PORT=port, PROXY_USER=username, PROXY_PASS=password, path="auth_chrome_extension.zip"))

        self.driver = webdriver.Chrome(service=Service(executable_path), chrome_options=chrome_options, desired_capabilities=capabilities)
        
        stealth(self.driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,   )

    def open_new_tab(self, url):
        self.driver.execute_script("window.open('');")
        self.driver.switch_to.window(self.driver.window_handles[-1])
        self.driver.get(url)

    def switch_to_tab(self, index):
        self.driver.switch_to.window(self.driver.window_handles[index])

In [ ]:
bot = SeleniumBot(executable_path='C:/Users/2me41/Downloads/chromedriver_win32(3)/chromedriver.exe', proxy_str="ni0us:9nc484th:169.197.83.74:19880")

#Verify the quality from: https://bot.sannysoft.com/
#verify ip with https://httpbin.org/ip

In [ ]:
bot.driver.get('https://wordcounter.net/')

In [ ]:
response = bot.driver.execute_script("""
    function run() {
        // Creating Our XMLHttpRequest object 
        var xhr = new XMLHttpRequest();
  
        // Making our connection  
        var url = 'https://wordcounter.net/json/english.json';
        xhr.open("GET", url, false);  // Set the third argument to false to make the request synchronous
  
        // Sending our request 
        xhr.send();
  
        // function execute after request is successful 
        if (xhr.readyState == 4 && xhr.status == 200) {
            console.log(xhr.responseText);
            return xhr.responseText;
        }
    }
    return run();
""")

In [ ]:
json.loads(response)